In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [3]:
# Шаг 1: Загрузка данных о времени

data = pd.read_csv('Time_Series_clients_2016-2019.csv')
data['date'] = pd.to_datetime(data['date'])

In [4]:
data.head()

,date,came
0,2016-01-10,1345
1,2016-01-17,2066
2,2016-01-24,1979
3,2016-01-31,1909
4,2016-02-07,1575


In [16]:
# Шаг 2: Тест Дики-Фуллера для проверки стационарности последовательного времени

result = adfuller(data['came'])
print("Результаты теста Дики-Фуллера :")
print(f"Статистика тестов : {result[0]}")
print(f"P-значение : {result[1]}")
print(f"Критические значения :")
for key, value in result[4].items():
    print(f"\t{key}: {value}")

Результаты теста Дики-Фуллера :
Статистика тестов : -1.293736019906016
P-значение : 0.632033197296583
Критические значения :
	1%: -3.47086589632514
	5%: -2.879330279087651
	10%: -2.57625492712671


In [18]:
# Шаг 3: Разложение последовательного времени

decomposition = seasonal_decompose(data['came'], model='additive', period=7)
trend = decomposition.trend
seasonal = decomposition.seasonal
residuals = decomposition.resid

In [19]:
from scipy.stats import boxcox
# Шаг 4: преобразование Бокса-Кокса и тест Дики-Фуллера

transformed_data, lambda_value = boxcox(data['came'])
transformed_result = adfuller(transformed_data)
print('Статистика теста Дики-Фуллера после преобразования :', transformed_result[0])
print('p-значение после преобразования :', transformed_result[1])
print('Критические значения после преобразования :', transformed_result[4])

Статистика теста Дики-Фуллера после преобразования : -1.4972692876443763
p-значение после преобразования : 0.5348617481593065
Критические значения после преобразования : {'1%': -3.470616369591229, '5%': -2.8792214018977655, '10%': -2.57619681359045}


In [20]:
# Шаг 5: Обучение модели ARIMA и прогнозирование

train_size = int(len(data) * 0.8)
train_data = data['came'][:train_size]
test_data = data['came'][train_size:]
model = ARIMA(train_data, order=(1, 1, 1))
model_fit = model.fit()
predictions = model_fit.predict(start=train_size, end=train_size + len(test_data) - 1)

In [21]:
# Шаг 6: Рассчитать показатели оценки

mae = mean_absolute_error(test_data, predictions)
mape = mean_absolute_percentage_error(test_data, predictions)
smape = 2 * np.mean(np.abs(predictions - test_data) / (np.abs(predictions) + np.abs(test_data)))

In [22]:
print("Показатели оценки :")
print(f"MAE : {mae}")
print(f"MAPE : {mape}")
print(f"SMAPE : {smape}")

Показатели оценки :
MAE : 124.97939652317024
MAPE : 0.09535299503206908
SMAPE : 0.08907347947124689
